<a href="https://colab.research.google.com/github/Kolawole-a2/Kola_Projects/blob/main/SEAS8416_AFOLABI_Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Setup and Imports

In [1]:
import os
import hashlib
import base64
import json


Step 2: Function to Create and Store a Salted Hash

In [2]:
def create_and_store_hash(user_input, filename='stored_hash.json'):
    # Step 1: Create a secure random 32-byte salt
    salt = os.urandom(32)

    # Step 2: Combine input with salt and hash using SHA-256
    hash_obj = hashlib.sha256(salt + user_input.encode())
    hashed_value = hash_obj.digest()

    # Step 3: Encode both for storage (make JSON-safe)
    salt_b64 = base64.b64encode(salt).decode()
    hash_b64 = base64.b64encode(hashed_value).decode()

    # Step 4: Store in a file
    with open(filename, 'w') as f:
        json.dump({'salt': salt_b64, 'hash': hash_b64}, f)

    # Step 5: Print results
    print("Salt (base64):", salt_b64)
    print("Hashed Value (base64):", hash_b64)


Step 3: Function to Update the Stored Hash (After Password Verification)

In [3]:
def update_hash_if_verified(old_password, new_password, filename='stored_hash.json'):
    if not os.path.exists(filename):
        print("Hash file does not exist.")
        return

    # Load stored salt and hash
    with open(filename, 'r') as f:
        data = json.load(f)

    stored_salt = base64.b64decode(data['salt'])
    stored_hash = base64.b64decode(data['hash'])

    # Hash old_password with stored_salt
    hash_check = hashlib.sha256(stored_salt + old_password.encode()).digest()

    if hash_check == stored_hash:
        print("Password verified successfully. Updating stored hash...")

        # Create new salt and hash for new password
        new_salt = os.urandom(32)
        new_hash = hashlib.sha256(new_salt + new_password.encode()).digest()

        # Store new salt and hash
        data = {
            'salt': base64.b64encode(new_salt).decode(),
            'hash': base64.b64encode(new_hash).decode()
        }

        with open(filename, 'w') as f:
            json.dump(data, f)

        print("Hash updated successfully.")
    else:
        print("Incorrect password. Hash not updated.")


***Step 4: Example Test Run ***

In [4]:
# Get user input
initial_password = input("Enter your initial password: ")
create_and_store_hash(initial_password)

# Try to update
old_pass = input("Enter your old password to change it: ")
new_pass = input("Enter your new password: ")
update_hash_if_verified(old_pass, new_pass)


Enter your initial password: Administrator123@
Salt (base64): SzfSiwoW4nIco8QJVObV07/D//u4qZtp0qnNJm4/NV4=
Hashed Value (base64): RHg2k70XOvElp6nOLbtI1Y02DyP0kwmM2vw56w/5mpw=
Enter your old password to change it: Administrator123@
Enter your new password: Admin456#
Password verified successfully. Updating stored hash...
Hash updated successfully.


***What the above Steps of Codes do ***

1. Generates a secure salt and hashes user's password with it.

2. Stores both the salt and the hash, safely encoded, in a JSON file.

3. Allows user to change the password only if they know the original one.

4. Uses SHA-256 (secure hashing) and random 32-byte salts (to prevent reuse or rainbow table attacks).

*******************************************************************************************************


**Why a Salted Hash Matters for Password Security**

When it comes to storing sensitive information like passwords, hashing alone isn't sufficient. Hashing is a one-way function that transforms a password into a fixed-length string, often using algorithms like SHA-256. However, if two users select the same password, their hashed values will be identical, which can expose a pattern and open the door to attacks like hash comparison and reuse (OWASP, 2023).

To address this, developers use what’s called a salt, which is a unique random string added to each password before hashing. Salting ensures that even if two users choose the same password, their resulting hashes will look entirely different. This practice helps prevent the use of rainbow tables, which are precomputed databases of common passwords and their corresponding hashes (Boneh and Shoup, 2020). By salting passwords, each hash becomes unique and much harder to crack, even if the underlying password is common.

Here’s a simplified example. Without a salt, the password “password123” would always hash to the same value. But with a random salt such as “X7aP@dK9,” the hash becomes unique and unrelated to other users’ hashes, even if they use the same password.

Salting is a simple yet highly effective strategy, and it's considered one of the foundational best practices in secure password storage today (OWASP, 2023).



**Zero-Knowledge Proofs in Authentication (Using Python)**

Zero-Knowledge Proofs (ZKPs) provide a powerful approach to authentication by enabling a user to prove they know a piece of information, like a password, without actually revealing what it is. This concept is especially valuable in scenarios where privacy and confidentiality are essential, such as in secure identity systems or privacy-focused financial platforms (Boneh and Shoup, 2020).

In Python, one promising tool for working with ZKPs is pySNARK, which lets developers write and verify mathematical proofs without revealing the underlying data (pySNARK, 2021). With this library, you can implement a system where the user proves they know their password or authentication secret, and the server verifies the proof without ever seeing the password itself.

This approach can significantly enhance traditional authentication methods. First, it improves privacy because the server does not directly retain or process sensitive data. Second, it strengthens security since the attacker cannot retrieve actual passwords or authentication material even if a system is compromised. Third, it simplifies compliance with privacy laws like the General Data Protection Regulation (GDPR), since less sensitive user data is stored and transmitted (OWASP, 2023).

For example, using pySNARK, you can prove that a password meets certain strength requirements or matches a hash without showing the password at any point during the verification process.



**References**

Boneh, D. and Shoup, V. (2020). A Graduate Course in Applied Cryptography. [online] Available at: https://toc.cryptobook.us/

OWASP Foundation. (2023). Password Storage Cheat Sheet. [online] Available at: https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html

pySNARK. (2021). GitHub Repository. [online] Available at: https://github.com/meilof/pysnark



****************************************************************************************************************************************************************


**Question 2:** Name one AES encryption mode that requires an Initial Vector (IV) and explain why it is considered more secure than other modes.  Name one AES encryption mode that does not require an IV.

****************************************************************************************************************************************************************

**Answer to Question 2:**

One commonly used AES encryption mode that requires an Initialization Vector (IV) is Cipher Block Chaining (CBC) mode. CBC enhances security by ensuring that identical plaintext blocks produce different ciphertext blocks, even when encrypted with the same key. This is achieved by applying an exclusive OR operation between each plaintext block and the previous ciphertext block before encryption. The first block uses an IV instead of a previous ciphertext block, which must be random and unpredictable for the encryption to remain secure (Ferguson et al. 2010). The use of an IV in CBC mode prevents attackers from identifying patterns in the data, which is a vulnerability in simpler modes such as ECB.

In contrast, Electronic Codebook (ECB) mode does not require an IV. While ECB is conceptually straightforward because each plaintext block is encrypted independently, it suffers from a major weakness. Identical plaintext blocks result in identical ciphertext blocks. This exposes structural patterns in the encrypted data, which can lead to information leakage, especially in image or file encryption scenarios (Stallings 2017).

For this reason, CBC mode is considered more secure than ECB because the inclusion of a properly managed IV introduces randomness. This helps defend against pattern analysis and replay attacks. However, it is important to note that IVs must never be reused with the same key in CBC mode, as doing so can undermine the encryption’s effectiveness.




**References**

Ferguson N., Schneier B., and Kohno T. (2010). Cryptography Engineering: Design Principles and Practical Applications. Wiley.
Stallings W. (2017). Cryptography and Network Security: Principles and Practice (Seventh Edition). Pearson.
